uitleg knn -> werking: buren
uitleg decision-tree-regr -> alg uitleg

Data set:
    split in train + test
    uitleg cross-val (uit cursus) + extend naar time series

WARD:
KNN -> cross met train (gridsearch gedoe) -> geeft knn met beste params
repeat for decision tree regr

vgl test err op test set + plot

conclusie: x beter dan y in deze situatie ...




voorspellings lijn over de tijd van test set


how do x models compare to time series data:
concl: knn wrs shit

## Question 3

**How does the decision tree regressor model compare to the k-nearest neighbour regressor model in terms of in- and out-of-sample error for time series forecasting?**

### Training data description

To investigate the research question, we use a Kaggle dataset containing many in-house measurements of temperatures and humanities, as well as weather data and appliances energy consumption. For our goal, we focus
on predicting the humidity in the kitchen (in percentages) based on the other measurements.

The data contains 15000 measurements from the period: 01/11/2016 until 24/04/2016. These are measurements of four months, of 105 different day. Each day contains 144 measure. This is one measure every ten minutes.<br>
Below, a table can be seen that shows the data we are given. All the features contain numbers, except the feature "date" which is of the type datetime. This must be taken into consideration when feeding a model with this data. Not every model is can handle the datetime, so we will adapt it.

| | Column | Data type |
| --- | --- | --- |
| 1  |  date |          datetime |
| 2  |  T1 - T9 |       float64 |
| 3  |  RH_1 - RH_9 |   float64 |
| 20 |  T_out |         float64 |
| 21 |  Press_mm_hg |   float64 |
| 22 |  RH_out |        float64 |
| 23 |  Windspeed |     float64 |
| 24 |  Visibility |    float64 |
| 25 |  Tdewpoint |     float64 |
| 26 |  rv1 |           float64 |
| 27 |  rv2 |           float64 |
| 28 |  lights |        int64 |
| 29 |  Appliances |    int64 |

### Initialisation

We set up some utility functions to create plots of the humidity over time and per hour and include the used datascience libraries.

In [ ]:
import warnings

import pandas as pd
import sklearn.base

warnings.filterwarnings('ignore')
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import RandomizedSearchCV
from time import time
import matplotlib.pyplot as plt


In [ ]:
def plot_months(ax: plt.Axes, column: pd.DataFrame, label: str) -> None:
	"""
	Plot a data column for two months
	:param ax: Pyplot axis
	:param column: data column to plot
	:param label: label for the plot
	:return: None
	"""
	ax.plot(column, label=label)
	ax.set_xlabel("Time")
	ax.set_ylabel("Kitchen humidity (%)")


def plot_hours(ax: plt.Axes, data: pd.DataFrame, column: str) -> None:
	"""
	Plot boxplots of the data column for every hour
	:param ax: Pyplot axis
	:param data: dataframe
	:param column: column inside dataframe to plot
	:return: None
	"""
	data.boxplot(column=[column], by='hour', ax=ax)
	ax.set_xlabel("Hours")
	ax.set_ylabel("Kitchen humidity (%)")

### Dataset preprocessing and exploring

We read in the relevant dataset columns. _RH_1_ is the column containing kitchen humidity values which we want to predict.

In [ ]:
# Columns to read in
columns = [
	"date",
	"lights",
	"T1", "RH_1",
	"T2", "RH_2",
	"T3", "RH_3",
	"T4", "RH_4",
	"T5", "RH_5",
	"T6", "RH_6",
	"T7", "RH_7",
	"T8", "RH_8",
	"T9", "RH_9",
	"T_out",
	"Press_mm_hg",
	"RH_out",
	"Windspeed",
	"Visibility",
	"Tdewpoint",
	"Appliances"
]

In [ ]:
# To make this command work, 1) make sure you pip installed kaggle and 2) have a kaggle.json file in the correct directory.
# More information on the kaggle.json file can be found here: https://www.kaggle.com/docs/api
!kaggle datasets download -d loveall/appliances-energy-prediction

from zipfile import ZipFile
zf = ZipFile('appliances-energy-prediction.zip')
zf.extractall('data-rq-3/') #save files in selected folder
zf.close()


# In case doing the kaggle.json setup is too much work, you can just download the csv manually from: https://www.kaggle.com/datasets/loveall/appliances-energy-prediction
# Just make sure the KAG_energydata_complete.csv file is then saved in the directory: data-rq-3/
data = pd.read_csv("data-rq-3/KAG_energydata_complete.csv", header=0, usecols=columns)
data

The table contains over 5 months worth of measurements taken every 10 minutes. To speed up the training and prediction process for the experiments we limit ourselves to the first 2 full months. We also resample the time series to contain hourly measurements to smooth out very harsh fluctuations in humidity.<br>
Since the models that we want to compare don't support datetime values, we split this column into three separate numerical month, day and hour columns. Using our existing knowledge about humidity, it is a reasonable assumption that these features 
might have a significant impact on the variable to be predicted.

In [ ]:
# Select the first two months of measurements
data = data[(data['date'] > '2016-02-01 00:00:00') & (data['date'] < '2016-04-01 00:00:00')]

# Split the date column into two month, day and hour columns
data['date'] = pd.to_datetime(data['date'])
data["month"] = pd.to_numeric(data["date"].dt.strftime("%m"))
data["day"] = pd.to_numeric(data["date"].dt.strftime("%d"))
data["hour"] = pd.to_numeric(data["date"].dt.strftime("%H"))

# Set the date column to be the index and resample the time series to hourly data
data = data.set_index('date')
data = data.resample("60T").mean()

We split the time series into a training and testing set by taking the first 75% of the data as training data and the last 25% as test data. No shuffling is done as we want to predict future values only.

In [ ]:
# Split training and test data
train_data, test_data = train_test_split(data, shuffle=False)
# Remove/add the humidity value to be predicted
X_train = train_data.drop(["RH_1"], 1)
y_train = train_data["RH_1"]
X_test = test_data.drop(["RH_1"], 1)
y_test = test_data["RH_1"]

We plot the kitchen humidity values over time available in our training set. We can see they range between 32.5 and 52.5% and are quite erratic at first sight at this scale. A general downward trend can be observed with a peak in the middle. The values after 15/03 seem to have made an unexpected jump.

In [ ]:
# Plot the training kitchen humidity over time
fig, ax = plt.subplots(figsize=(20, 10))
plot_months(ax, train_data["RH_1"], "Actual humidity")
ax.set_title("Kitchen humidity over time (training dataset)")

To investigate the influence of the passing of the day in hours on the humidity values, we create an hourly boxplot over the training data. We observe that the variation in humidity in general stays similar but rises in the afternoon and evening. Exceptionally higher humidity values are observed at 19h.

In [ ]:
# Plot the training kitchen humidity variation every hour
fig, ax = plt.subplots(figsize=(20, 10))
plot_hours(ax, train_data, "RH_1")
fig.suptitle("Kitchen humidity per hour (training dataset)")

### Training, Testing and Validating

We set up the experiment by creating multiple functions to help us train and evaluate models and to parameter selection using cross-validation.<br>
The first function aids in measuring the training and test score and error. As this is a regression problem we use the root-mean-square error
$$E = \sqrt{\dfrac{\sum_{n=0}^N (y_n - h(x_n))^2}{N}}$$
The score is then calculated as follows
$$1-\dfrac{\sum_{n=0}^N (y_n - h(x_n))^2}{\sum_{n=0}^N (y_n - \bar{y})^2}$$
for mean $\bar{y}$. The best possible score is thus 1 but the score may also be negative when the model performs worse than predicting the average $\bar{y}$ over the training set. This function then also makes some comparative plots.<br>
The second function creates and evaluates a baseline model with normalisation where the default hyperparameters are used.<br>
The third function does a randomized grid search over a parameter grid by sampling a certain number of configurations at random without replacement. The grid search makes use of cross-validation. However, for time series prediction we can't use the standard k-fold cross-validation, so we use the method explained above. In order to not bias our comparison of the two models, the parameter selection explores an equal amount of sample configurations for both.
The best parameter set is then used to train a model on the full training set. Finally, this model is evaluated.

In [ ]:
def evaluate_model(model: sklearn.pipeline.Pipeline,
                   X_train: pd.DataFrame, X_test: pd.DataFrame, y_train: pd.DataFrame, y_test: pd.DataFrame) -> None:
	"""
	Evaluate the model by calculating the training and test score and RMSE, plotting the predicted and actual humidity values for the test dataset and full dataset
	and the hourly difference between the predicted and actual humidity values
	:param model: model to evaluate
	:param X_train: training features
	:param X_test: test features
	:param y_train: training labels
	:param y_test: test labels
	:return: None
	"""
	# Calculate the training and test score and RMSE
	print(f"Training score: {model.score(X_train, y_train):.3f}, Test score: {model.score(X_test, y_test):.3f}")
	print(
		f"Training RMSE: {mean_squared_error(y_train, model.predict(X_train), squared=False):.3f} Test RMSE: {mean_squared_error(y_test, model.predict(X_test), squared=False):.3f}")

	# Give predictions
	test_data["predictions"] = model.predict(X_test)
	test_data["error"] = test_data["RH_1"] - test_data["predictions"]
	train_data["predictions"] = model.predict(X_train)

	# Plot the predicted and actual humidity values for the test dataset
	fig, ax = plt.subplots(figsize=(20, 10))
	plot_months(ax, test_data["RH_1"], "Actual humidity")
	plot_months(ax, test_data["predictions"], "Predicted humidity")
	ax.set_title("Kitchen humidity over time (test dataset)")
	plt.legend()
	plt.plot()

	# Plot the predicted and actual humidity values for the full dataset
	fig, ax = plt.subplots(figsize=(20, 10))
	plot_months(ax, test_data["RH_1"], "Actual humidity (test dataset)")
	plot_months(ax, test_data["predictions"], "Predicted humidity (test dataset)")
	plot_months(ax, train_data["RH_1"], "Actual humidity (training dataset)")
	plot_months(ax, train_data["predictions"], "Predicted humidity (training dataset)")
	ax.set_title("Kitchen humidity over time (full dataset)")
	plt.legend()
	plt.plot()

	# Plot the hourly difference between the predicted and actual humidity values
	fig, ax = plt.subplots(figsize=(20, 10))
	plot_hours(ax, test_data, "error")
	ax.set_title("Kitchen humidity deviation per hour (test dataset)")
	fig.suptitle("")
	plt.plot()


def create_baseline_model(model: DecisionTreeRegressor | KNeighborsRegressor, X_train: pd.DataFrame,
                          y_train: pd.DataFrame, X_test: pd.DataFrame,
                          y_test: pd.DataFrame) -> sklearn.pipeline.Pipeline:
	"""
	Create a baseline model with normalisation and evaluate it
	:param model: model to use
	:param X_train: training features
	:param y_train: training labels
	:param X_test: test features
	:param y_test: test labels
	:return: the trained pipeline
	"""
	model = make_pipeline(StandardScaler(), model)
	model.fit(X_train, y_train)

	evaluate_model(model, X_train, X_test, y_train, y_test)

	return model


def randomize_grid_search(model: sklearn.pipeline.Pipeline, parameters: dict, X_train: pd.DataFrame,
                          y_train: pd.DataFrame, X_test: pd.DataFrame, y_test: pd.DataFrame,
                          iterations: int) -> RandomizedSearchCV:
	"""
	Randomly search for the best hyperparameters for the given model in a given parameter grid using time series cross validation, fit the best model on the full training set and evaluate it
	:param model: pipeline to use
	:param parameters: parameter grid to search in
	:param X_train: training features
	:param y_train: training labels
	:param X_test: test features
	:param y_test: test labels
	:param iterations: how many samples to do
	:return: the tuned and trained model
	"""
	start = time()
	tuner = RandomizedSearchCV(model, parameters, n_iter=iterations, n_jobs=-2, cv=TimeSeriesSplit(), verbose=1,
	                           scoring="neg_root_mean_squared_error", random_state=0)

	tuner.fit(X_train, y_train)
	print(f"{((time() - start) / 60):.2f} minutes passed")
	print(f"Best cross-validation RMSE: {-tuner.best_score_:.3f}")
	print(f"Best parameters: {tuner.best_params_}")

	evaluate_model(tuner.best_estimator_, X_train, X_test, y_train, y_test)

	return tuner

### The Decision Tree Regressor

We run the experiment first on a baseline decision tree regressor without configuration.

In [ ]:
# Create a baseline decision tree regressor and evaluate it
model = DecisionTreeRegressor(random_state=0)

model = create_baseline_model(model, X_train, y_train, X_test, y_test)

The top plot visualises the true and predicted humidity values in function of the time on the test set over approximately two weeks. We that a general upward trend is correctly predicted by the model based on the features even though this trend wasn't really present in the training data. A test score of 0.479 is already obtained with a relatively low error in humidity of 1.551%. This can be recognised in the graph. Although the model is able to predict some smaller fluctuations, the actual bigger peaks cannot. However, sometimes similar peaks are predicted where they do not occur.<br>
A score of 1 and RMSE of 0 indicate that the tree is fitting the training data perfectly. This can also be observed in the second graph. There is good chance the baseline model is actually overfitting the data and picking up noisy features. This might be resolved by applying model selection in the next step.<br>
When looking at the third graph displaying the deviation between the actual and predicted humidity in the test set for every hour, we observe that in general the percentage is overestimate slightly but underestimated for the peak at 19h.

We now select the best parameters for the decision tree regressor based on the training data using time series cross-validation. We sample 1000 configurations from the grid where we consider the following hyperparameters: splitting method,
tree depth, minimum leaf samples, amount of features to consider at every split and the amount of leaf nodes.

In [ ]:
# Do a randomised grid search for the best parameters and evaluate the selected model
parameters = {
	"decisiontreeregressor__splitter": ["best", "random"],  # Split using the squared error criterion or randomly
	"decisiontreeregressor__max_depth": [4, 8, 10, 12, 14, None],  # Maximum depth of the tree
	"decisiontreeregressor__min_samples_leaf": [1, 4, 8, 10, 12],  # Minimum number of samples required a leaf
	"decisiontreeregressor__max_features": [None, "log2", "sqrt"],
	# Number of features to consider when looking for the best split
	"decisiontreeregressor__max_leaf_nodes": [20, 30, 40, 50, 60, None],  # Maximum number of leaf nodes
}

model = randomize_grid_search(model, parameters, X_train, y_train, X_test, y_test, iterations=1000)

By looking at the cross-validation RMSE, we can notice that a simpler, more regularised decision tree is able to generalise better on a dataset of this size. This is accomplished by reducing the tree depth, samples per leave and amount of leaves.<br>
When this model is trained on the full training set it achieves an increased test score of 0.554 and reduced humidity error  of 1.434%. This is a decrease in RMSE of more than 0.1%. Looking at the graph of the test set only, this is perceived by more conservative predictions. The model predicts more flat surfaces (a result of returning the average humidity in a leaf) and is able to recognise potential peaks. However, it avoids predicting wrong extreme humidity values in contrast to the baseline model.<br>
The regularised model doesn't fit the training data perfectly anymore but comes really close with a training score of  0.949. In the second graph this is recognisable by the fact that the predicted training humidities are less erratic.<br>
When looking at the error in humidity per hour, we now see that the humidities are not overestimated as much anymore although the underestimation at 19h is still present with a big variance in deviation.

### The K-Nearest Neighbours Regressor

The previous experiment is repeated for the k-nearest neighbours regressor. We first establish a baseline model again without configuration.

In [ ]:
# Create a baseline K-Nearest Neighbors regressor and evaluate it
model = KNeighborsRegressor()

model = create_baseline_model(model, X_train, y_train, X_test, y_test)

As is clear from the first graph, this baseline nearest neighbours model is much more susceptible to the noise present in the training data. As a consequence the model's test score is only 0.040, barely better than predicting the mean training humidity. The RMSE is above 2% humidity. The predictions on the test set contain multiple big unwarranted peaks and are less in line with the general upward trend.<br>
Looking at the predictions on the training set, we see that this model doesn't fit the humidity values perfectly (not surprising given that the default amount of neighbours is 5) but comes really close. The outlying humidity values are not fitted.<br>
Similar to the decision tree regressor, the third graph shows that the median hourly prediction is too low while the humidity peaks around 19h are underestimated.

In a similar fashion, we do parameter selection by randomly sampling 1000 parameter configurations from the grid and validating them with cross-validation. We consider the parameters relating to the amount of neighbours, the weighting of the neighbours, the power parameter for the Minkowski metric used, the type of tree datastructure to find the neighbours and the leave size within this tree.

In [ ]:
# Do a randomised grid search for the best parameters and evaluate the selected model
parameters = {
	"kneighborsregressor__n_neighbors": [1, 2, 5, 10, 20, 50, 60],  # Number of neighbours to consider
	"kneighborsregressor__algorithm": ["auto", "ball_tree", "kd_tree", "brute"],
	# The algorithm and datastructure to find the neigbours
	"kneighborsregressor__weights": ["uniform", "distance"],  # The weight function used in the prediction
	"kneighborsregressor__leaf_size": [1, 2, 5, 10, 15, 20],  # The leaf size of the tree
	"kneighborsregressor__p": [1, 2, 3, 4],  # The power of the Minkowski distance metric
}

model = randomize_grid_search(model, parameters, X_train, y_train, X_test, y_test, iterations=1000)

The best parameter set obtained through selection has a root-mean-square error of 2.089% humidity. Surprisingly, this configured model does fit the training set perfectly which can be observed in the second graph. This can be explained by the chosen parameter settings. Instead of uniformly weighting the importance of the neighbours, the influence of the points on the prediction is weighted by the inverse of their distance. This is combined with a Manhattan-distance instead of Euclidean distance and the amount of neighbours to be considered is increased.<br>
In this way, the tuned model achieves a slightly increased score of 0.089, still not much better than predicting the mean. But by looking at the graph of the testing set, we see that less extreme jumps in humidity are now avoided although large fluctuations remain that are not always present in reality. This could be explained by the many jumps present in the dataset for which the Manhattan-distance is close to the test features.<br>
The humidity deviation per hour is very similar to the baseline model, although the variance in deviation has reduced slightly.

### Conclusion

We can conclude that for this dataset of in-house measurements and weather data, a decision tree regressor model selected through cross-validation generalises better to future predictions of humidity percentages than a k-nearest neighbour regressor selected through the same process. <br>
The nature and inner workings of the two regression machine learning models express themselves in their future predictions.
The predictions of the decision tree regressor were smoother with fewer peaks and contained multiple flat surfaces when the depth of the tree was limited. The k-nearest neighbour regressor on the other hand, was much more susceptible to large fluctuations and noise and it's predictions on the test set, although not correct, resembled more the actual values in the training set.<br>
The nature of time series predictions in which we mainly try to extrapolate from the training data in the past, makes it so that similar measurements are less relevant than in a standard regression problem. This could potentially explain the poor testing accuracy of the nearest neighbours model.<br>
We have observed that for both regression algorithms, the untuned baseline model's testing accuracy could be increased through parameter selection with time-series cross-validation. In the case of the decision tree this resulted in a more regularised model that overfitted less. For the k-nearest neighbour model, this was not as obviously visible as the training error actually disappeared while the testing predictions became more stable too.<br>
In conclusion, our experiments indicated that the decision tree regressor is a better fit for this type of time series forecasting problem for this particular dataset and behaviour of predicted variable. The prediction accuracy of the model could potentially be improved by preprocessing the data further and removing redundant features. Cross-validation is not only important to model selection but can help in evaluating these rather manual decisions too.